In [34]:
import tensorflow as tf
import kerastuner as kt
import os
import shutil
import string

## Useful Constants

* A = number of different genotypic alleles
* P = number of different phenotypes
* D = dropout rate
* l2 = L2 regularization rate

In [14]:
A = 1
P = 1

## Network Architecture
Outputs:

1. Input: N x A x 3
1. Dense Layer w/ ReLU: N x A // 3
1. Dropout(D): N x A // 3
1. Dense Layer w/ ReLU: N x P * 3
1. Dense Layer w/ Sigmoid: N x P

In [15]:
def make_DNN_model(d1, dropout, d2, l2):
    regularizer = tf.keras.regularizers.L2(l2)
    return tf.keras.Sequential(layers=[
        tf.keras.layers.InputLayer(input_shape=(A, 3)),
        tf.keras.layers.Dense(d1, activation=tf.nn.relu, kernel_regularizer=regularizer),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(d2, activation=tf.nn.relu, kernel_regularizer=regularizer),
        tf.keras.layers.Dense(P, activation=tf.nn.sigmoid, kernel_regularizer=regularizer)
    ], name='nn_model')

## Model Training Parameters
#### Loss Function
Binary Cross-Entropy + L2 Regularization
#### Optimizer
Adam
#### Metric
Binary Accuracy

In [16]:
def compile_model(model, optimizer):
    model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

## Hyperparameter Tuning

In [39]:
batch_size = 1
epochs = 10
use_multiprocessing=True
validation_split = 0.2

def get_tunable_DNN_model(hp, d1_min, d1_max, d2_min, d2_max, d_step):
    d1_units = hp.Int('d1_units', min_value=d1_min, max_value=d1_max, step=d_step)
    d2_units = hp.Int('d2_units', min_value=d2_min, max_value=d2_max, step=d_step)
    dropout = hp.Choice('dropout', values=[0.2, 0.5, 0.8])
    l2 = hp.Choice('l2', values=[0.01, 0.005, 0.001])
    lr = hp.Choice('lr', values=[0.01, 0.005, 0.001])
    model = make_DNN_model(d1_units, dropout, d2_units, l2)
    compile_model(model, tf.keras.optimizers.Adam(lr))
    return model

def get_tuner(model_builder):
    shutil.rmtree('hp_tuning')
    return kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='hp_tuning', project_name='initial_model')

## Search Space

In [18]:
d1_min = max(A // 4, 1)
d1_max = A * 2
d2_min = P * 2
d2_max = P * 4
d_step = 1

## Get Training Data

In [19]:
# TODO
def get_data(N, A, P):
    genotypes = tf.stack([tf.constant([[1, 0, 0]] * A, dtype=tf.float32)] * N)
    phenotypes = tf.stack([tf.constant([1] * P, dtype=tf.float32)] * N)
    return genotypes, phenotypes

## Hyperparameter Tuning Call

In [36]:
tuner = get_tuner(lambda hp: get_tunable_DNN_model(hp, d1_min, d1_max, d2_min, d2_max, d_step))
tuner.search_space_summary()
tuner.search(*get_data(10, A, P), epochs=epochs, validation_data=get_data(2, A, P))
tuner.results_summary()

Trial 5 Complete [00h 00m 04s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hp_tuning\initial_model
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
d1_units: 2
d2_units: 2
dropout: 0.8
l2: 0.001
lr: 0.01
Score: 1.0
Trial summary
Hyperparameters:
d1_units: 1
d2_units: 4
dropout: 0.5
l2: 0.005
lr: 0.01
Score: 1.0
Trial summary
Hyperparameters:
d1_units: 2
d2_units: 3
dropout: 0.2
l2: 0.01
lr: 0.001
Score: 0.6666666666666666
Trial summary
Hyperparameters:
d1_units: 2
d2_units: 3
dropout: 0.5
l2: 0.005
lr: 0.01
Score: 0.6666666666666666
Trial summary
Hyperparameters:
d1_units: 2
d2_units: 2
dropout: 0.2
l2: 0.001
lr: 0.001
Score: 0.6666666666666666


## Optimal Model

In [37]:
optimal = [2, 2, 0.8, 0.001, 0.01]
model = make_DNN_model(optimal[0], optimal[2], optimal[1], optimal[3])
model.compile(optimizer=tf.keras.optimizers.Adam(optimal[4]), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

## Train Model

In [40]:
x, y = get_data(1000, A, P)
history = model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, validation_split=validation_split, use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1)

Epoch 1/10
800/800 [==============================] - 1s 892us/step - loss: 0.1645 - accuracy: 0.9739 - val_loss: 0.0075 - val_accuracy: 1.0000
Epoch 2/10
800/800 [==============================] - 1s 773us/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 3/10
800/800 [==============================] - 1s 731us/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 4/10
800/800 [==============================] - 1s 722us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 5/10
800/800 [==============================] - 1s 735us/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 6/10
800/800 [==============================] - 1s 732us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 7/10
800/800 [==============================] - 1s 877us/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy:

## Test Model

In [41]:
test_x, test_y = get_data(10, A, P)
history = model.evaluate(x=test_x, y=test_y, use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1)

1/1 [==============================] - 0s 159ms/step - loss: 4.9914e-04 - accuracy: 1.0000
